In [44]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

import torch
from torch.utils.data import DataLoader

import os
import pandas as pd
from tqdm import tqdm
import csv

In [10]:
CACHE_DIR = "../cache_dir"

In [11]:
tokenizer = AutoTokenizer.from_pretrained("gpt2-xl", padding_side="left", cache_dir=CACHE_DIR)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained("gpt2-xl", cache_dir=CACHE_DIR)
model.to("mps")
model.eval()

ds = load_dataset("gbharti/finance-alpaca", cache_dir=CACHE_DIR)

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [46]:
save_path = "unlabeled.csv"
write_steps = 5
if not os.path.exists(save_path):
    with(open(save_path, "w")) as f:
        f.write("prompt,response_a,response_b\n")

with open(save_path, "a", newline='') as f:
    writer = csv.writer(f, quoting=csv.QUOTE_MINIMAL)
    write_batch = []
    dl = DataLoader(ds["train"], batch_size=4, shuffle=False)
    for step, batch in tqdm(enumerate(dl)):
        tokens = tokenizer(batch["instruction"], padding=True, return_tensors="pt").to(model.device)
        prompt_len = tokens.input_ids.shape[1]
        out_a = model.generate(**tokens, max_new_tokens=16, do_sample=True, pad_token_id=50256)
        out_b = model.generate(**tokens, max_new_tokens=16, do_sample=True, pad_token_id=50256)
        prompts = tokenizer.batch_decode(out_a[:,:prompt_len], skip_special_tokens=True) 
        responses_a = tokenizer.batch_decode(out_a[:,prompt_len:], skip_special_tokens=True)
        responses_b = tokenizer.batch_decode(out_b[:,prompt_len:], skip_special_tokens=True)

        # Batch out outputs so we minimize the amount of file writes.
        # Lines are wrapped in quotes so the commas can go in the csv.
        write_batch += list(zip(prompts, responses_a, responses_b))
        if (step + 1) % write_steps == 0:
            writer.writerows(write_batch)
            write_batch.clear()


32it [05:23, 10.10s/it]


KeyboardInterrupt: 

In [48]:
df = pd.read_csv(save_path)
df

,prompt,response_a,response_b
0,"For a car, what scams can be plotted with 0% f...","The list is quite large, and there are many m...",\n\n\nIs there a certain type of buyer that is...
1,Why does it matter if a Central Bank has a neg...,\n\nIt all depends on how that negative rate i...,\n\nIf the Central Bank were to adopt a zero-i...
2,Where should I be investing my money?,\n\nIf you are looking for a guaranteed return...,If you're looking to invest in a specific mar...
3,Specifically when do options expire?,\n\nGenerally in this release there are four t...,\n\nDo we need to renew these options?\n\nWhat...
4,Negative Balance from Automatic Options Exerci...,We can't pay off mortgages. We need to find a...,Consider the stock is currently more than $25...
...,...,...,...
115,"UK student loans, early repayment/avoiding fur...",and the need to make extra payments for stude...,\n\nThe aim of the LDCs is to provide the stud...
116,Should I purchase a whole life insurance polic...,My wife and I have made life policies but we ...,\n\nI am just about to go off to college and w...
117,Why are wire transfers and other financial ser...,\n\nThe explanation is that the Canadian finan...,\n\nWhy in the first decade of the 21st centur...
118,Stochastic Oscillator for Financial Analysis,"""\n\nDerek Hirsch, PhD, ""Quantitative Analysis...",. The R package dplyr is available for downloa...
